In [1]:
from src.config import FilePaths
from src.document_chunking import chunk_markdown_file
from src.enums import Party

party = Party.VVD

# Short markdown files are formatted and trimmed; no introduction, summary etc
short_markdown_file = FilePaths.short_markdown_dir / f"{party}_short.md"

with open(short_markdown_file, 'r', encoding='utf-8') as file:
    markdown_string = file.read()

chunks = chunk_markdown_file(markdown_string)
example_chunk = chunks[20]
example_chunk

Document(metadata={'Hoofdstuk': 'Missie 1: Radicale economische groei', 'Sectie': 'Groei is verbonden met alles wat het leven mooi maakt'}, page_content='Wat de komende jaren extra aandacht vergt is de industrie. Een vertrek van de (basis)industrie maakt ons afhankelijker van andere landen. De industrie staat bijvoorbeeld aan de basis van de productie van nachtkijkers voor het leger, infuuszakken voor het ziekenhuis en de schoenen waar je dagelijks op loopt. Ook zorgen ze voor innovatie. We hebben tijdens de coronacrisis gezien waarom het belangrijk is dat zulke zaken in Nederland of Europa kunnen worden gemaakt.')

In [2]:
import pandas as pd
from langchain_core.documents import Document

def chunks_to_dataframe(chunks: list[Document]) -> pd.DataFrame:
    """
    Converts a list of LangChain Document objects into a pandas DataFrame,
    extracting specified metadata fields.
    """
    data = []
    
    for chunk in chunks:
        metadata = chunk.metadata
        page_content = chunk.page_content

        # Extract metadata fields with a default value of None or an empty string
        hoofdstuk = metadata.get('Hoofdstuk', "")
        sectie = metadata.get('Sectie', "")
        subsectie = metadata.get('Subsectie', "")

        data.append({
            'Hoofdstuk': hoofdstuk,
            'Sectie': sectie,
            'Subsectie': subsectie,
            'Text': page_content,
        })
        
    return pd.DataFrame(data)

In [3]:
chunks_to_dataframe(chunks)[:50]

,Hoofdstuk,Sectie,Subsectie,Text
0,Missie 1: Radicale economische groei,,,De sterkste economie van Europa
1,Missie 1: Radicale economische groei,Dit is wat we gaan doen om van Nederland de st...,,- Alle seinen op groen voor groei: we verlagen...
2,Missie 1: Radicale economische groei,Dit is wat we gaan doen om van Nederland de st...,,- Een streep door knellende regels: we hantere...
3,Missie 1: Radicale economische groei,Dit is wat we gaan doen om van Nederland de st...,,- Investeren in groei van de toekomst: met een...
4,Missie 1: Radicale economische groei,Dit is wat we gaan doen om van Nederland de st...,,- Steun voor ondernemers en zelfstandigen: we ...
5,Missie 1: Radicale economische groei,Groei is verbonden met alles wat het leven moo...,,Ons land heeft altijd vooropgelopen in de were...
6,Missie 1: Radicale economische groei,Groei is verbonden met alles wat het leven moo...,,"Of de biotech in Leiden, die op dat vlak tot d..."
7,Missie 1: Radicale economische groei,Groei is verbonden met alles wat het leven moo...,,Nederland is een land van ondernemers. We zijn...
8,Missie 1: Radicale economische groei,Groei is verbonden met alles wat het leven moo...,,Nederland telt dit jaar relatief meer failliss...
9,Missie 1: Radicale economische groei,Groei is verbonden met alles wat het leven moo...,,Er moet wat gebeuren om dit soort vaderlandse ...


In [7]:
def format_embedding_content(chunk: Document) -> str:
    metadata = chunk.metadata
    page_content = chunk.page_content

    chapter_title = metadata.get("Hoofdstuk")
    if not chapter_title:
        raise ValueError("No chapter title found")

    section_title = metadata.get("Sectie", "")
    subsection_title = metadata.get("Subsectie", "")

    embedding_content = f"{chapter_title}\n{section_title}\n{subsection_title}\n{page_content}"

    return embedding_content

In [8]:
embedding_content = format_embedding_content(example_chunk)

print(embedding_content)

Missie 1: Radicale economische groei
Groei is verbonden met alles wat het leven mooi maakt

Wat de komende jaren extra aandacht vergt is de industrie. Een vertrek van de (basis)industrie maakt ons afhankelijker van andere landen. De industrie staat bijvoorbeeld aan de basis van de productie van nachtkijkers voor het leger, infuuszakken voor het ziekenhuis en de schoenen waar je dagelijks op loopt. Ook zorgen ze voor innovatie. We hebben tijdens de coronacrisis gezien waarom het belangrijk is dat zulke zaken in Nederland of Europa kunnen worden gemaakt.
